In [1]:
!unzip jangod.zip

Archive:  jangod.zip
   creating: dataset/
   creating: dataset/front_palm/
  inflating: dataset/front_palm/img_0.jpg  
  inflating: dataset/front_palm/img_1.jpg  
  inflating: dataset/front_palm/img_2.jpg  
  inflating: dataset/front_palm/img_3.jpg  
  inflating: dataset/front_palm/img_4.jpg  
  inflating: dataset/front_palm/img_5.jpg  
  inflating: dataset/front_palm/img_6.jpg  
  inflating: dataset/front_palm/img_7.jpg  
  inflating: dataset/front_palm/img_8.jpg  
  inflating: dataset/front_palm/img_9.jpg  
  inflating: dataset/front_palm/img_10.jpg  
  inflating: dataset/front_palm/img_11.jpg  
  inflating: dataset/front_palm/img_12.jpg  
  inflating: dataset/front_palm/img_13.jpg  
  inflating: dataset/front_palm/img_14.jpg  
  inflating: dataset/front_palm/img_15.jpg  
  inflating: dataset/front_palm/img_16.jpg  
  inflating: dataset/front_palm/img_17.jpg  
  inflating: dataset/front_palm/img_18.jpg  
  inflating: dataset/front_palm/img_19.jpg  
  inflating: dataset/front_palm/im

In [2]:
!ls

cnn.keras    hand_gesture_recog_cnn.ipynb  photo	     predict.py
dataset      hand_landmarker.task	   photo.bak
dataset.zip  jangod.zip			   photu_capture.py


In [25]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

In [26]:
BATCH_SIZE = 32
IMG_SIZE = 300

In [27]:
train_ds = tf.keras.utils.image_dataset_from_directory(
  "dataset",
  validation_split=0.1,
  subset="training",
  seed=69420,
  batch_size=BATCH_SIZE)

val_ds = tf.keras.utils.image_dataset_from_directory(
  "dataset",
  validation_split=0.1,
  subset="validation",
  seed=69420,
  batch_size=BATCH_SIZE)

complete_ds = tf.keras.utils.image_dataset_from_directory("dataset")

Found 357 files belonging to 7 classes.
Using 322 files for training.
Found 357 files belonging to 7 classes.
Using 35 files for validation.
Found 357 files belonging to 7 classes.


In [28]:
train_ds.class_names

['back_palm',
 'fist',
 'front_palm',
 'index_finger',
 'left_thumb',
 'peace',
 'right_pinky']

In [29]:
# Caching dataset and shiii (idk tf docs say this)
AUTOTUNE = tf.data.AUTOTUNE

train_ds = train_ds.cache().shuffle(1000).prefetch(buffer_size=AUTOTUNE)
val_ds = val_ds.cache().prefetch(buffer_size=AUTOTUNE)
complete_ds = complete_ds.cache().shuffle(1000).prefetch(buffer_size=AUTOTUNE)

In [30]:
# Resize and rescaling stuff
resize_and_rescale = tf.keras.Sequential([
  layers.Resizing(IMG_SIZE, IMG_SIZE),
  layers.Rescaling(1./255)
])

In [31]:
# augmentation stuff
data_augmentation = tf.keras.Sequential([
  layers.RandomRotation((-0.2,0.2), fill_mode="nearest"),
  layers.RandomBrightness(0.2, value_range=(0.0,1.0)),
  layers.RandomContrast(0.2, value_range=(0.0,1.0)),
  # layers.RandomShear(x_factor=0.1, y_factor=0.1, fill_mode="nearest"),
  layers.RandomTranslation(height_factor=0.2, width_factor=0.2, fill_mode="nearest"),
  layers.RandomZoom(0.2, fill_mode="nearest"),
])

In [32]:
model = tf.keras.Sequential([
  resize_and_rescale,
  data_augmentation,

  layers.Conv2D(16, (3,3), activation="relu"),
  layers.MaxPooling2D(pool_size=(2, 2)),
  layers.Conv2D(32, (3,3), activation="relu"),
  layers.MaxPooling2D(pool_size=(2, 2)),
  layers.Conv2D(64, (3,3), activation="relu"),
  layers.MaxPooling2D(pool_size=(2, 2)),
  layers.Conv2D(128, (3,3), activation="relu"),
  layers.MaxPooling2D(pool_size=(2, 2)),
  layers.Conv2D(256, (3,3), activation="relu"),
  layers.MaxPooling2D(pool_size=(2, 2)),

  layers.Flatten(),
  layers.Dense(128, activation="relu"),
  layers.Dropout(0.1),
  layers.Dense(64, activation="relu"),
  layers.Dropout(0.1),
  layers.Dense(32, activation="relu"),
  layers.Dropout(0.1),
  layers.Dense(7, activation="relu"),
])

In [33]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])
model.summary()

Model: "sequential_8"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ sequential_6 (Sequential)       │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ sequential_7 (Sequential)       │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_14 (Conv2D)              │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_14 (MaxPooling2D) │ ?                      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_15 (Conv2D)              │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_15 (MaxPooling2D) │ ?                      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_16 (Conv2D)              │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_16 (MaxPooling2D) │ ?                      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_17 (Conv2D)              │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_17 (MaxPooling2D) │ ?                      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_18 (Conv2D)              │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_18 (MaxPooling2D) │ ?                      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_3 (Flatten)             │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_11 (Dense)                │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_7 (Dropout)             │ ?                      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_12 (Dense)                │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_8 (Dropout)             │ ?                      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_13 (Dense)                │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_9 (Dropout)             │ ?                      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_14 (Dense)                │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

In [34]:
model.fit(complete_ds, epochs=100)

Epoch 1/100
12/12 ━━━━━━━━━━━━━━━━━━━━ 9s 547ms/step - accuracy: 0.1124 - loss: 1.9520
Epoch 2/100
12/12 ━━━━━━━━━━━━━━━━━━━━ 7s 572ms/step - accuracy: 0.1354 - loss: 1.9455
Epoch 3/100
12/12 ━━━━━━━━━━━━━━━━━━━━ 7s 570ms/step - accuracy: 0.1430 - loss: 1.9459
Epoch 4/100
12/12 ━━━━━━━━━━━━━━━━━━━━ 7s 543ms/step - accuracy: 0.1284 - loss: 1.9462
Epoch 5/100
12/12 ━━━━━━━━━━━━━━━━━━━━ 6s 521ms/step - accuracy: 0.1605 - loss: 1.9451
Epoch 6/100
12/12 ━━━━━━━━━━━━━━━━━━━━ 7s 537ms/step - accuracy: 0.1670 - loss: 1.9348
Epoch 7/100
12/12 ━━━━━━━━━━━━━━━━━━━━ 6s 530ms/step - accuracy: 0.2283 - loss: 1.8973
Epoch 8/100
12/12 ━━━━━━━━━━━━━━━━━━━━ 6s 521ms/step - accuracy: 0.2518 - loss: 1.8936
Epoch 9/100
12/12 ━━━━━━━━━━━━━━━━━━━━ 7s 548ms/step - accuracy: 0.1499 - loss: 1.9403
Epoch 10/100
12/12 ━━━━━━━━━━━━━━━━━━━━ 7s 603ms/step - accuracy: 0.1321 - loss: 1.9402    
Epoch 11/100
12/12 ━━━━━━━━━━━━━━━━━━━━ 7s 581ms/step - accuracy: 0.1706 - loss: 1.8879
Epoch 12/100
12/12 ━━━━━━━━━━━━━━━━━━

In [35]:
model.save("cnn.keras")